#Summarizing Game Reviews with LLMs

## Importando os pacotes

In [ ]:
!pip install -q openai
!pip install -q langchain
!pip install -q tiktoken
!pip install -q requests
!pip install -q bs4
!pip install -q langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import openai
import tiktoken
import requests
import re
import pandas as pd
import numpy as np
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from bs4 import BeautifulSoup
from langdetect import detect

## Extraindo os dados do Metacritic

In [ ]:
# verifica se a próxima página de reviews existe
def nextPage(soup):
  if(soup.find(rel="next") == None):
    return False

  return True

In [ ]:
def web_scraping(link):
  v = link
  v = v[32:]
  div = v.find('/')
  platform = v[:div]
  v = v[div+1:]
  div = v.find('/')
  game = v[:div]
  v = v[div+1:]
  div = v.find('/')
  reviewer = v[:]

  print(f'Web Scraping: {game} of {platform} ({reviewer})')
  review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}
  page = 0

  url = link + '?page='+str(page)
  user_agent = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
  response  = requests.get(url, headers = user_agent)
  soup = BeautifulSoup(response.text, 'html.parser')

  while((reviewer == 'user-reviews' and (nextPage(soup) or page == 0)) or (reviewer == 'critic-reviews' and page == 0)):
    # print(f'Page {page}')

    for review in soup.find_all('div', class_='review_content'):
      if review == None:
        break

      if(reviewer == 'user-reviews'):
        if review.find('span', class_='blurb blurb_expanded'):
          rv = review.find('span', class_='blurb blurb_expanded').text
        else:
          if(review.find('div', class_='review_body').find('span') is None):
            rv = ''
          else:
            rv = review.find('div', class_='review_body').find('span').text
      else:
        if(review.find('div', class_='review_body') is None):
          rv = ''
        else:
          rv = review.find('div', class_='review_body').text

      try:
        language = detect(rv)
      except:
        continue

      if(language == 'en'):
        try:
          if(reviewer == 'user-reviews'):
            name = review.find('div', class_='name').find('a').text
            rating = int(review.find('div', class_='review_grade').find_all('div')[0].text)
          else:
            name = review.find('div', class_='source').find('a').text
            rating = int(review.find('div', class_='review_grade').find_all('div')[0].text)
            rating = round(rating/10)

          date = review.find('div', class_='date').text

        except:
          continue

        review_dict['name'].append(name)
        review_dict['date'].append(date)
        review_dict['rating'].append(rating)
        review_dict['review'].append(rv)

    page += 1
    url = link+'?page='+str(page)
    user_agent = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
    response  = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')

  data = pd.DataFrame(review_dict)
  data['review'] = [" ".join(r.split()) for r in data['review']]
  return data

In [ ]:
link = "https://www.metacritic.com/game/playstation-3/the-last-of-us/user-reviews"
data = web_scraping(link)
data

Web Scraping: the-last-of-us of playstation-3 (user-reviews)


,name,date,rating,review
0,brad3ngunn,"Jun 14, 2013",10,I don't know what I can really say about this ...
1,Badonkadonk,"Jun 15, 2013",10,"I wouldn't call this game perfect, but it's pr..."
2,jakenelson,"Jun 18, 2013",10,I don't think I have ever played a game like t...
3,Qishere,"Jun 14, 2013",10,"You should realize right from the beginning, y..."
4,journeyfreak787,"Jun 14, 2013",10,This game truly is phenomenal. Naughty Dog pro...
...,...,...,...,...
2336,Aleklar,"May 18, 2023",10,This game is perfect This game is perfect This...
2337,OMGitsTHEMEMI,"May 18, 2023",10,This game is just perfection from the amazingl...
2338,AlesiLeite,"May 30, 2023",9,A game that gives you the feeling of immersing...
2339,Diven,"May 21, 2023",10,"Nice game really nice to play , waiting for pa..."


## Métodos de Particionamento

**Simple Splitting**

In [ ]:
# Algorithm 1: Equal Size Partitioning
def simple_splitting(T,P=2000):
  # Data: Text T, Partition size P
  # Return: Partitions of text T

  tokens = re.split(' ', T)
  partitions = [' '.join(tokens[i:i+P]) for i in range(0,len(tokens),P)]

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. Five word sentences are fine."
simple_splitting(text, 4)

['This sentence has five',
 'words. Here are five',
 'more words. Five word',
 'sentences are fine.']

**Sliding Window**

In [ ]:
# Algorithm 2: Sliding Window Partitioning
def sliding_window_partitioning(T,w=2000,s=1800):
  # Data: Text T, Window size w, Step size s
  # Result: Partitions of text T using sliding windows

  tokens = re.split(' ', T)
  partitions = [' '.join(tokens[i:i+w]) for i in range(0,len(tokens),s)]

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. Five word sentences are fine."
sliding_window_partitioning(text, 4, 3)

['This sentence has five',
 'five words. Here are',
 'are five more words.',
 'words. Five word sentences',
 'sentences are fine.']

**Natural Boundary Splitting**

In [ ]:
def sentence_boundary(t):
  if re.match(r"\w*[.!?]", t):
    return True
  return False

In [ ]:
def review_boundary(t):
  if re.match(r"\n\n", t):
    return True
  return False

In [ ]:
# Algorithm 3: Boundary-based Partitioning
def boundary_based_partitioning(T,m=2000,B=review_boundary):
  # Data: Text T, Maximum partition size m, Boundary function B(t)
  # Result: List of partitions P

  tokens = re.split(' ', T)
  partitions = []
  p = []

  for t in tokens:
    if re.match(r"\w+[.!?]*", t):
      p.append(t)

    if(B(t) or len(p) == m):
      partitions.append(' '.join(p))
      p = []

  if(p != []):
    partitions.append(' '.join(p))

  return partitions

In [ ]:
# Algorithm 4: Multiple Boundaries Partitioning
def multiple_boundaries_partitioning(T,m=2000,B=review_boundary):
  # Data: Text T, Maximum partition size m, Boundary function B(t)
  # Result: List of partitions P

  tokens = re.split(' ', T)
  partitions = []
  p = []

  for t in tokens:
    if re.match(r"\w+[.!?]*", t):
      p.append(t)

    if(B(t)):
      if(len(p) >= m or len(p)+1 > m):
        partitions.append(' '.join(p))
        p = []

  if(p != []):
    partitions.append(' '.join(p))

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
boundary_based_partitioning(text, 4, sentence_boundary)

['This sentence has five',
 'words.',
 'Here are five more',
 'words.',
 'Five word sentences are',
 'fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
boundary_based_partitioning(text, 4, review_boundary)

['This sentence has five',
 'words. Here are five',
 'more words.',
 'Five word sentences are',
 'fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
multiple_boundaries_partitioning(text, 4, sentence_boundary)

['This sentence has five words.',
 'Here are five more words.',
 'Five word sentences are fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
multiple_boundaries_partitioning(text, 4, review_boundary)

['This sentence has five words. Here are five more words.',
 'Five word sentences are fine.']

## Técnicas de Amostragem

**Uniform Sampling**

In [ ]:
# Algorithm 15: Uniform Sampling
def uniform_sampling(C,n):
  # Data: Collection C, Sample size n
  # Result: Sample S

  return C.sample(n=n)

In [ ]:
uniform_sampling(data,10)

,name,date,rating,review
95,BigBiggo,"Jan 4, 2014",9,The characters and how their relationships are...
45,J-P-M,"Jun 14, 2013",10,i am amazed at Sony for putting out a must own...
825,Xelima,"Jun 29, 2013",10,"The last of us is awesome, and so amazingly am..."
1734,Ghob,"Nov 6, 2013",10,"Story: 10/10, it was so amazing to see the plo..."
1620,ffs,"Sep 21, 2021",8,Simply put it's a pretty good mixture of narra...
1018,AddictedPanda,"Jun 21, 2013",7,"There are good games, and then there's Medal o..."
1194,dantah,"Feb 5, 2019",0,The Last of Us is the very definition of an ov...
1382,TheBrokenGamer,"Jun 3, 2014",10,The Last of Us PlayStation 3 This review is ba...
1140,UserReviewsLUL,"Feb 15, 2020",5,"Good movie, boring game naughty dog only make ..."
863,tarsus,"Jul 28, 2013",10,I'd like to start by saying that this is a phe...


**Stratified Sampling with Known Labels**

In [ ]:
# Algorithm 16: Stratified Sampling with Known Class Labels
def stratified_sampling_known_class_labels(C,n):
  # Data: Collection C with class labels L, Sample size n,
  # Result: Sample S

  S = pd.DataFrame(columns=['name','date','rating','review'])
  for L in range(0,11):
    cL = data[data['rating'] == L]
    nL = round(n*len(cL)/len(C))
    sL = cL.sample(n=nL)
    S = pd.concat([S,sL])

  return S

In [ ]:
stratified_sampling_known_class_labels(data,10)

,name,date,rating,review
1063,Dunge,"Jun 15, 2013",8,"Amazing game, great levels and story events, b..."
1561,slappysfan,"Jun 23, 2013",9,This game lived up to my expectations and was ...
1552,tombo,"Dec 21, 2013",10,"I loved ""The Last of Us"" so much that I didn't..."
1675,fabiolfpinto,"Jul 5, 2013",10,The Last Of Us is a masterpiece. Troy and Ashl...
439,Jaredwheeler,"Jun 17, 2013",10,I just finished this game on hard (about to st...
638,hollowfication,"Jun 19, 2013",10,i had just finished this game and i have say t...
1490,EwokSmuggler,"Nov 9, 2020",10,"Story is Amazing, Gameplay is Great, and Facti..."
992,ekolet,"Jul 11, 2013",10,Best game ever created. I've never attached an...
2335,Henrique3669,"May 12, 2023",10,Just for the record that this is the best game...


## Resumindo as avaliações usando LLMs

In [ ]:
def summarize(R, S=stratified_sampling_known_class_labels, P=multiple_boundaries_partitioning, C='map_reduce'):
  # Data: Reviews R (DataFrame), Sampling Function S, Partitioning Function P, Chain Type C
  # Result: Summarization of Reviews R

  llm = OpenAI(
    model_name='text-davinci-003',
    openai_api_key='OPENAI_API_KEY',
    max_tokens=1024)

  prompt_template = """Following the reviews bellow, write a small summary:

  {text}

  SUMMARY OF THE REVIEWS:"""

  PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

  if(len(R) > 100):
    R = S(R,100)

  reviews = ''
  for r in R['review']:
    reviews += r + ' \n\n '

  texts = P(reviews)
  docs = [Document(page_content=t) for t in texts]

  if(C == 'map_reduce'):
    chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT)
    response = chain.run(docs)


  elif(C == 'refine'):
    chain = load_summarize_chain(llm, chain_type="refine", refine_prompt=PROMPT)
    response = chain.run(docs)

  else:
    print("Got unsupported chain type! Running with 'map_reduce'")
    chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT)
    response = chain.run(docs)

  response = " ".join(response.split())
  return response

In [ ]:
summarize(data,uniform_sampling,simple_splitting,'map_reduce')

'The Last of Us is a critically acclaimed post-apocalyptic game from Naughty Dog that has been lauded for its emotionally engaging story, thrilling gameplay, exquisite level design, and realistic violence. It has stunning graphics, a great soundtrack, and a unique online mode, and its characters, dialogue, and voice acting have been praised as being compelling and deeply engaging. Despite some minor flaws, the game is seen as an incredible experience and is considered one of the best games of all time. It is a must-play for anyone with a PS3.'

In [ ]:
summarize(data,uniform_sampling,simple_splitting,'refine')

'The Last of Us is a highly acclaimed video game that features realistic visuals, tough but rewarding combat, and well-developed characters, all in an immersive world with excellent pacing. It has been praised for its attention to detail, unique approach to stealth, and its realism. It is highly recommended for players who want to experience a thrilling adventure.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,'map_reduce')

'The Last of Us is a highly-praised game from developer Naughty Dog that has been praised for its emotionally engaging and immersive story, detailed characters, excellent graphics and audio, and challenging yet enjoyable gameplay. Despite some critiques about the lack of environment traversal and repetitiveness, the majority of reviews have found the game to be an incredibly enjoyable and rewarding experience.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,'refine')

'The game has been described as a masterpiece and a classic in the videogame world with stunning graphics, an engaging story and believable characters. Its gameplay allows for different strategies and playstyles. It also has an upgrade system and crafting system which add to the atmosphere and gameplay. The soundtrack and voice acting are superb and it has been praised for being one of the best PS3 games with limitless potential. Although the game has been highly praised, some have noted that it is not for everyone due to its atmosphere and violence.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,'map_reduce')

'The Last of Us is an acclaimed post-apocalyptic game developed by Naughty Dog with a gripping story, emotionally developed characters, and intense gameplay. It follows Joel and Ellie on their journey across the United States, as they scavenge for resources and fight against both zombies and hostile humans. The game has been widely praised for its immersive story, cinematic experience, and satisfying combat. Despite criticisms of AI and repetitiveness, The Last of Us has been hailed as one of the greatest games of all time.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,'refine')

'The Last of Us is widely considered to be one of the best video games ever made, being praised for its story, characters, atmosphere, graphics, and sound. It is a post-apocalyptic survival game with an excellent story arc, pacing, and characters that draw the player in. The game features intense moments of panic and tranquility and has an overall immersive and beautiful aesthetic. It is a game that is often compared to literature and other forms of art due to its strong story and characters. Many reviewers found the game to be a masterpiece, with an engrossing story and characters, though it was noted to have some minor flaws such as its overreliance on disc readers and a few glitches.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels,simple_splitting,'map_reduce')

"The Last of Us is an acclaimed action-adventure survival horror game developed by Naughty Dog and published by Sony Computer Entertainment. It is praised for its exceptional storytelling, beautiful graphics, and strong characters. It has a unique atmosphere with realistic violence, stealth mechanics, crafting capabilities, and upgradable weapons. It has received overwhelmingly positive reviews and is considered one of the best games of its generation, with Nolan North's voice acting being particularly lauded. It is an immersive experience that will make players feel connected to the characters and the game's events, making it a must-play."

In [ ]:
summarize(data,stratified_sampling_known_class_labels,simple_splitting,'refine')

'The Last of Us is an amazing experience that needs to be had by everyone! It has a rich and engaging setting, beautiful graphics, a deep and heartfelt story, and unique and fun multiplayer. It has become one of the best games of the year due to its stunning visuals, intense gameplay, and memorable characters. The only complaints are minor audio bugs and character glitching. It is a masterpiece and highly recommended to everyone.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels,sliding_window_partitioning,'map_reduce')

'The Last of Us is a critically acclaimed game from Naughty Dog that is widely considered to be one of the best games of its generation. It features a gripping story with well-crafted characters, intense stealth-based combat, and immersive environments. It also poses thought-provoking moral questions that challenge the player and makes them think about the consequences of their actions. Despite some flaws in the multiplayer and AI, this is a must-play game that has earned its place as a classic.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels,sliding_window_partitioning,'refine')

'The Last of Us is a highly acclaimed PlayStation 3 game from Naughty Dog that features amazing graphics, stunning sound effects, captivating story, well-written characters, and excellent gameplay. It has been praised as one of the best games of its generation for its ability to evoke emotion and tension while still maintaining a sense of story and plot. Players are immersed in a post-apocalyptic world and given a thrilling experience as they control Joel and Ellie on their journey.'

In [ ]:
summarize(data, stratified_sampling_known_class_labels, multiple_boundaries_partitioning,'map_reduce')

'The Last of Us is a critically acclaimed Playstation 3 game by Naughty Dog that follows the story of Joel and Ellie in a post-apocalyptic world. It is praised for its immersive story, detailed characters, realistic graphics, and realistic violence. It has been acclaimed for its cinematic opening, its emotional depth, and its connection between gameplay and story. It has been praised for its length and quality, though its AI and multiplayer mode have been criticized. It has been praised for its poignant themes and its impact on the gaming industry, and is widely considered to be one of the best games of this generation.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels,multiple_boundaries_partitioning,'refine')

"The Last of Us is considered one of the best games ever made, with great graphics, an immersive gameplay, and a compelling story. Even though the reviewer does not own a PlayStation 3 and initially found the controller to be difficult to use, he was so drawn in by the game that he decided to buy it and play it for 14 hours straight. He found the fighting system to be varied and enjoyable, and the story to be one of the best he's ever experienced in a video game. The visuals and atmosphere were also praised, as well as the realistic gameplay and its challenge. The reviewer ultimately gave the game a 9.5/10."